In [1]:
import pandas as pd
import numpy as np
import csv
import scipy
import sys
import os
import subprocess

In [2]:
DIR = 'c'

In [15]:
os.environ['PATH']

'C:\\Users\\alsmi\\.conda\\envs\\py38;C:\\Users\\alsmi\\.conda\\envs\\py38\\Library\\mingw-w64\\bin;C:\\Users\\alsmi\\.conda\\envs\\py38\\Library\\usr\\bin;C:\\Users\\alsmi\\.conda\\envs\\py38\\Library\\bin;C:\\Users\\alsmi\\.conda\\envs\\py38\\Scripts;C:\\Users\\alsmi\\.conda\\envs\\py38\\bin;C:\\ProgramData\\Miniconda3\\condabin;C:\\ProgramData\\Miniconda3;C:\\ProgramData\\Miniconda3\\Library\\mingw-w64\\bin;C:\\ProgramData\\Miniconda3\\Library\\usr\\bin;C:\\ProgramData\\Miniconda3\\Library\\bin;C:\\ProgramData\\Miniconda3\\Scripts;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0;C:\\Windows\\System32\\OpenSSH;C:\\Program Files\\Microsoft VS Code\\bin;C:\\Program Files\\Git\\cmd;C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common;C:\\Program Files\\Docker\\Docker\\resources\\bin;C:\\ProgramData\\DockerDesktop\\version-bin;C:\\MinGW\\bin;C:\\Users\\alsmi\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Users\\alsmi\\Minicon

## Характеристики задачи

In [3]:
def find_features_tasks(df):
    '''
    Находит следующие характеристики для задачи:
    
    Число периодических программ
    Общая сложность периодических программ
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    
    Args:
        df(pd.DataFrame): Данные о постановке задачи построения статико-динамического
                          расписания
    Returns:
        int: число программ,
        int: общая сложность программ, 
        int: размер цикла планирования, 
        int: число разделов, 
        float: загрузка процессора этой задачей, 
        int: число интервалов
    '''
    Time = 1
    periods = df['period'].as_matrix()
    runtimes = df['complexity'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    print("Num of partitions: ", num_part)
    for x in periods:
        Time = NOK(Time, x)
        
    counts = np.array([Time/x for x in periods])
    
    all_runtime = sum(runtimes[i]*counts[i] for i in range(len(df)))
    print("Complexity: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in periods:
        y = x
        while(y <= Time):
            intervals.add(y);
            y += x;
            
    num_of_intervals = len(intervals) - 1;
    print("Runtime: ",Time)
    load = all_runtime / Time
    print('Load: ', load)
    print("Intervals: ", num_of_intervals)
    return tuple((num_tasks,all_runtime, Time, num_part, load, num_of_intervals))

def find_features_works(df):
    '''
    Находит следующие характеристики для задачи
    Число задач
    Общее время выполнения
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    '''
    
    starts = df['start'].as_matrix()
    finishes = df['finish'].as_matrix()
    runtimes = df['runtime'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    #print("Num of partitions: ", num_part)

    Time = np.max(finishes)   
    #counts = np.array([Time/x for x in periods])
    
    all_runtime = np.sum(runtimes)
    
    #print("All_runtime: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in starts:
        intervals.add(x)
    for x in finishes:
        intervals.add(x)          
    num_of_intervals = len(intervals) - 1;
    
    #print("All time: ",Time)
    
    load = all_runtime / Time
    
    average_runtime = all_runtime/ num_tasks
    #print('Load: ', load)
    #print("Число интервалов: ", num_of_intervals)
    return tuple((num_tasks, average_runtime, num_part, load, num_of_intervals))
      
    

In [4]:
def NOD(a, b):
    while a != 0 and b != 0:
        if a > b:
            a %= b
        else:
            b %= a
    return a + b


def NOK(a, b):
    return a*b / NOD(a, b)

def NOKA(a):
    first = a[0]
    for i in range(1, len(a)):
        second = a[i]
        first = first*second / NOD(first, second)
    
    return first


## Генерация по распределению  

In [4]:
def create_distrib(num_tasks, max_period = 10, mode='normal', mul_period=10, part=None, load=1.):
    '''
    Создается таблица разделы х задачи, в ячейках указывается время работы, как часть от периода.
    И задается такая же табличка для периодов.
    Можно указать загрузку.

    num_tasks - вектор числа задач по разделам

    mode - normal - равномерное распределени
    '''
    if (mode == 'normal'):
        num_part = len(num_tasks)
        max_tasks = max(num_tasks)
        all_tasks = sum(num_tasks)
        distrib = np.zeros((num_part, max_tasks))
        period = np.zeros((num_part, max_tasks))
        for i in range(num_part):
            for j in range(num_tasks[i]):
                period[i][j] = int(np.random.randint(1, max_period + 1)) * mul_period
                distrib[i][j] = load*1./all_tasks
        return distrib, period
    elif (mode == 'one_part'):
        num_part = num_tasks.shape[0]
        max_tasks = num_tasks.max()
        all_tasks = num_tasks.sum() - 1
        distrib = np.zeros((num_part, max_tasks))
        period = np.zeros((num_part, max_tasks))
        for i in range(num_part):
            for j in range(num_tasks[i]):
                period[i][j] = int(np.random.randint(1, max_period + 1)) * mul_period
                distrib[i][j] = load*(1.-part)/all_tasks
        period[0][0] = max_period * mul_period
        distrib[0][0] = part * load
        return distrib, period

def create_tasks_by_distrib(task_distrib, periods):
    '''
    Create one task for static-dynamic shedule
    task_distrib - nparray
    first axis - partition, second axis - distribution of time for every
    periods - periods for tasks
    '''
    df = pd.DataFrame(columns=['partition', 'runtime', 'period'])
    for i in range(task_distrib.shape[0]):
        for j in range(task_distrib.shape[1]):
            if (task_distrib[i][j] == 0):
                break
                
            d = np.array([0, 0, 0], dtype=int)
            d[0] = i + 1
            d[2] = periods[i][j]
            d[1] = task_distrib[i][j] * d[2]
            if (d[1] <= 1.):
                d[1] = 1.
            df.loc[len(df)] = d
    return df
    

example_distrib = np.array([[0.1, 0.3, 0.2],
                            [0.1, 0, 0],
                            [0.2, 0 ,0]])
example_periods = np.array([[10, 20, 40],
                            [50, 0, 0],
                            [100, 0 ,0]])

def create_test_sample_dist(num_tasks, DIR, mul_period=10, mode='normal', part=None, max_period = 10, load=1., time_to_switch = 1):
    '''
    num_tasks - список массивов
    Создает len(num_tasks) файлов с именами testi.tsv, где i - номер задачи
    В файл meta.tsv помещается информация об этих задачах
    Возвращает строку с именами файлов, записанными через запятую
    '''

    os.mkdir(DIR)
    with open(DIR+'/TEST.txt', 'w') as file_output:
        df_f =  pd.DataFrame(columns=['num_tasks','all_runtime', 'Time', 'num_part', 'load', 'num_of_intervals', 'time_to_switch'])
        for x in range(num_tasks.shape[0]):
            file = DIR+"/test" + str(x) + ".tsv"
            file_output.write(file+' '+str(time_to_switch)+'\n');
            print (num_tasks[x])
            distr, period = create_distrib(num_tasks[x], mul_period=mul_period, mode=mode, part=part, max_period=max_period, load=load)
            df = create_tasks_by_distrib(distr, period)
            df.to_csv(file, sep ="\t")

            features = find_features_tasks(df)       
            node = np.empty((len(features))+1)
            for i in range(len(features)):
                node[i] = features[i]
            node[len(features)] = time_to_switch
            df_f.loc[len(df_f)] = node 
        
    df_f.to_csv(DIR+"/meta.tsv", sep="\t")

tasks = np.array([[6, 2, 3],
                  [1, 2, 4],
                  [1, 2]])

create_test_sample_dist(tasks, "TestXIIIIII", mul_period=1000, mode='normal', max_period=10, load=0.7)

---

# Считывание систем

Всего есть Systems/system2 и Systems/system4

In [5]:
system_path = os.path.join(DIR, 'Systems', 'system2')
system2 = pd.read_csv(system_path, sep='\t', index_col=[0])

In [6]:
system_path = os.path.join(DIR, 'Systems', 'system4')
system4 = pd.read_csv(system_path, sep='\t', index_col=[0])

In [7]:
system_path = os.path.join(DIR, 'Systems', 'system8')
system8 = pd.read_csv(system_path, sep='\t', index_col=[0])

In [8]:
system8

,Performance,Options,Cost
0,4,o1,10
1,1,o2;o1,3
2,2,o2,4
3,3,o2;o1,8
4,4,o1,10
5,1,o2;o1,3
6,2,o2,4
7,3,o2;o1,8


---

# Периодические работы

## Равномерное распределение параметров задач

In [9]:
def create_tasks_random(tasks_bound, n_part, periods, load, system):
    '''
    Create one task for static-dynamic shedule с периодическими интервалами 
    по равномерному распределению
    Args:
        n_tasks(int): Число периодических программ в задаче
        n_part(int): число разделов в задаче
        periods(list(int)): возможные значения для периодов программ
        load(float): приблизительная нагрузка полученной задачи
        system(pd.DataFrame): Система в виде датафрейма
    
    Returns:
        pd.DataFrame: Постановка задачи в табличном формате с колонками
                      'partition', 'complexity', 'period', 'functionalities','cost'
    '''
    # Интервал планирования
    time_circle = NOKA(periods)
    
    # процессоры
    procs = list(system.index)
    
    # Разделы
    all_parts = np.array(range(1, n_part + 1))
    
    # готовим знание о системе
    options = set()
    for _, row in system.iterrows():
        options = options.union(set(row.Options.split(';')))
    # каждому процессору его перфоманс приписываем
    
    perf = {}
    for proc in procs:
        perf[proc] = system.iloc[proc].Performance * time_circle
        
    # Привяжем рандомно разделы к процессорам и свяжем опции
    bound = {}
    tasks = {}
    d_part_options = {}
    for part in all_parts:
        r = np.random.randint(len(procs))
        bound[part] = procs[r]
        # Определяем число задач в разделе
        tasks[part] = np.random.randint(tasks_bound[0], tasks_bound[1])
        d_part_options[part] = set(system.iloc[procs[r]].Options.split(';'))
        d_part_options[part] = ';'.join(d_part_options[part])
    #print( 'Распределение разделов:', bound)
    
    # Считаем число разделов по процессорам
    tasks_proc = {}
    for i,task in tasks.items():
        key = bound[i]
        if key in tasks_proc:
            tasks_proc[key].add(i)
        else:
            tasks_proc[key] = set({i})
        
    # Считаем процент загрузки для каждого раздела 
    part_load = {}
    for proc, parts in tasks_proc.items():
        a = np.random.sample(len(parts))
        a = a/a.sum()
        for i, part in enumerate(parts):
            part_load[part] = a[i]
    #print('Загрузка раздела:', part_load)
    
    df_all = pd.DataFrame()
    
    np.random.shuffle(all_parts)
    for part in all_parts:
        proc_cur = bound[part]
        performance = perf[proc_cur]
        rows = []
        cur_part_load = part_load[part]
        tasks_cur = tasks[part]
        a = np.random.sample(tasks_cur)
        a = a/a.sum()
        for i in range(tasks_cur):
            row = []
            np.random.shuffle(periods)      
            row.append(performance*load*a[i]*cur_part_load // (time_circle // periods[0]) )
            row.append(periods[0])
            rows.append(row)
        df = pd.DataFrame(rows, columns=['complexity', 'period'])
        df['partition'] = part
        df_all = df_all.append(df)
        
        
    df_all['left'] = 0
    df_all['right'] = df_all['period']
    df_all['options'] = df_all['partition'].map(d_part_options)
    df_all['complexity'] = df_all['complexity'].astype(int)
    return df_all[['partition','complexity','period','left','right','options']]
    

In [10]:
df_test = create_tasks_random(tasks_bound=(2,5), n_part=5, periods=[500, 1000, 2000, 10000], load=0.7, system=system2)

In [11]:
df_test

,partition,complexity,period,left,right,options
0,2,884,10000,0,10000,o2;o1
1,2,80,2000,0,2000,o2;o1
2,2,77,500,0,500,o2;o1
3,2,127,2000,0,2000,o2;o1
0,5,26,2000,0,2000,o2;o1
1,5,67,2000,0,2000,o2;o1
0,1,7,500,0,500,o2;o1
1,1,15,500,0,500,o2;o1
2,1,11,2000,0,2000,o2;o1
0,4,375,10000,0,10000,o2;o1


In [501]:
df_test.to_csv('C++/TestsHeterogenes/test_1.csv', sep='\t')

 -----------------
# Обычные работы


## Равномерное распределение

In [107]:
def create_works_plan(max_time, delta, num_of_works, num_of_part, load, c, n_proc=1, mul=1):
    '''
    Создание планируемого расписания  
    '''
    df = pd.DataFrame(columns=['partition', 'runtime', 'start', 'finish'])
    for j in range(n_proc):
        current_time = 0  
        for i in range(int(num_of_works/n_proc)):
            d = np.array([0, 0, 0, 0], dtype=int)
            d[1] = np.random.randint(1,max_time)*mul
            d[0] = np.random.randint(0, num_of_part/n_proc) + j*(num_of_part/n_proc) + 1
            d[2] = max(current_time - np.random.randint(delta)*mul, 0)
            d[3] = current_time + d[1] + np.random.randint(delta)*mul
            current_time += c + d[1]*1.0/load
            df.loc[len(df)] = d
    return df

In [112]:
df_test = create_works_plan(max_time=10, delta=100, num_of_works=37, num_of_part=7, load=1, c=1)

In [113]:
df_test

,partition,runtime,start,finish
0,1,2,0,20
1,7,9,0,62
2,7,4,0,113
3,1,9,0,104
4,2,5,0,83
5,2,4,0,55
6,6,7,0,48
7,1,2,46,62
8,6,1,0,79
9,3,6,22,135


In [114]:
def create_works_random(num_works, num_part, max_time):
    '''
    равномерное распределение
    для работ
    
    '''
    df = pd.DataFrame(columns=['partition', 'runtime', 'start', 'finish'])
    for i in range(num_works):
        d = np.array([0, 0, 0, 0], dtype=int)
        d[0] = np.random.randint(1, num_part + 1)
        d[2] = np.random.randint(1, max_time + 1)
        d[3] = np.random.randint(d[2], max_time + 1)
        d[1] = np.random.randint(d[2], d[3] + 1) - d[2]
        df.loc[len(df)] = d
    return df

In [117]:
df_test = create_works_random(num_works=37, num_part=7, max_time=100)

In [61]:
df_test

,partition,complexity,period,left,right,options
0,7,1754.0,2000,0,2000,o2;o1
1,7,379.0,500,0,500,o2;o1
0,9,68.0,1000,0,1000,o2
1,9,373.0,1000,0,1000,o2
2,9,125.0,500,0,500,o2
0,8,359.0,500,0,500,o2;o1
1,8,327.0,1000,0,1000,o2;o1
0,11,319.0,1000,0,1000,o2;o1
1,11,443.0,1000,0,1000,o2;o1
0,2,448.0,500,0,500,o2


---

# Обработка решения

In [17]:
def get_time_perfomance(file):
    time = 0.
    perfomance = 0.
    with open(file, 'r') as f:
        str_ = f.readline(); # Пропускаем заголовок
        str_ = f.readline();
        time = float(str_.split(',')[0])
        perfomance = float(str_.split(',')[2])
    return time, perfomance

In [18]:
def result_to_df(files):
    df = pd.DataFrame(columns=['Time_alg','Effectiveness'])
    for file in files:
        node = np.array([0,0], dtype=float)
        node[0], node[1] = get_time_perfomance(file)
        df.loc[len(df)] = node
    return df

In [19]:
def save(name='', fmt='png'):
    pwd = os.getcwd()
    iPath = './pictures/{}'.format(fmt)
    if not os.path.exists(iPath):
        os.mkdir(iPath)
    os.chdir(iPath)
    plt.savefig('{}.{}'.format(name, fmt), fmt='png')
    os.chdir(pwd)
    #plt.close()

# Проведение тестирования

Генерация тестов на лету будет (без повторения тогда получится)

In [26]:
n_iter = 10
configs = []
configs.append((3, [500, 1000, 2000],(2,5), system2, 0.6))
configs.append((5, [500, 1000, 2000],(2,5), system2, 0.7))
configs.append((7, [500, 1000, 2000],(2,5), system2, 0.8))

configs.append((5, [500, 1000, 2000],(2,5), system4, 0.6))
configs.append((7, [500, 1000, 2000],(2,5), system4, 0.7))
configs.append((9, [500, 1000, 2000],(2,5), system4, 0.8))

configs.append((9, [500, 1000, 2000],(2,5), system8, 0.6))
configs.append((11, [500, 1000, 2000],(2,5), system8, 0.7))
configs.append((13, [500, 1000, 2000],(2,5), system8, 0.8))

In [27]:
%%time
rows = []
for config in configs:
    n_part = config[0]
    periods = config[1]
    tasks_bound = config[2]
    system = config[3]
    load = config[4]
    for i in range(n_iter):
        df_test = create_tasks_random(tasks_bound=tasks_bound, n_part=n_part, periods=periods, load=load, system=system)
        df_test.to_csv(f'{DIR}/TestsHeterogenes/test.csv', sep='\t')
        result = subprocess.call([f'./{DIR}/shedulerH', '1', f'{DIR}/TestsHeterogenes/test.csv', f'{DIR}/Systems/system{system.shape[0]}' ,'schedule'])
        print(result)
        time, performance = get_time_perfomance(f'{DIR}/TestsHeterogenes/test.res')
        row = (n_part, periods, tasks_bound, system.shape[0], load, time, performance)
        rows.append(row)

0
0
0
0
0
0
0
0
0
Wall time: 1min 10s


In [28]:
df_res = pd.DataFrame(rows, columns=['Число разделов', 'Периоды', 'Количество задач', 'Система', 'Загрузка', 'Время', 'Эффективность'])

In [30]:
df_res

,Число разделов,Периоды,Количество задач,Система,Загрузка,Время,Эффективность
0,3,"[1000, 500, 2000]","(2, 5)",2,0.6,2.040000,1.0
1,5,"[500, 2000, 1000]","(2, 5)",2,0.7,2.997000,1.0
2,7,"[2000, 1000, 500]","(2, 5)",2,0.8,3.940000,1.0
3,5,"[1000, 500, 2000]","(2, 5)",4,0.6,4.514000,1.0
4,7,"[2000, 1000, 500]","(2, 5)",4,0.7,5.851000,1.0
5,9,"[1000, 500, 2000]","(2, 5)",4,0.8,6.781000,1.0
6,9,"[1000, 2000, 500]","(2, 5)",8,0.6,10.200000,1.0
7,11,"[500, 1000, 2000]","(2, 5)",8,0.7,16.143999,1.0
8,13,"[2000, 1000, 500]","(2, 5)",8,0.8,16.066999,1.0


In [31]:
df_res.groupby(['Система', 'Загрузка', 'Число разделов'])[['Время','Эффективность']].mean()

Время  Эффективность
Система Загрузка Число разделов                          
2       0.6      3                2.040000            1.0
        0.7      5                2.997000            1.0
        0.8      7                3.940000            1.0
4       0.6      5                4.514000            1.0
        0.7      7                5.851000            1.0
        0.8      9                6.781000            1.0
8       0.6      9               10.200000            1.0
        0.7      11              16.143999            1.0
        0.8      13              16.066999            1.0